In [ ]:
import numpy as np
import pandas as pd
import scipy
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline
from ggplot import *
# for maps
from mapboxgl.utils import *
from mapboxgl.viz import *
import geojson
# for text analysis
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from inflector import Inflector, English
inf = Inflector(English)

In [ ]:
file_path = './camb-pb-data/Participatory_Budgeting_Ideas_Submitted_by_Community_Members.csv'
pb_data = pd.read_csv(file_path)

In [ ]:
pb_data.head()

In [ ]:
pb_data['Committee'] = pb_data['Committee'].apply(lambda x: x.strip())

In [ ]:
pb_data.groupby('Committee').size()

In [ ]:
def rename_committees(committee):
    # code to rename committees
    if committee == 'Environment' or committee == 'Environment, Health & Safety' or committee == 'Health, Environment & Safety':
        return 'Environment, Health & Safety'
    elif committee == 'Parks & Recreation' or committee == 'Parks, Recreation & Education':
        return 'Parks, Recreation & Education'
    elif committee == 'Youth & Education' or committee == 'Youth & Technology':
        return 'Youth Education & Technology'
    elif committee == 'Streets, Sidewalks & Transit' or committee == 'Streetsmarts':
        return 'Streets, Sidewalks & Transit'
    elif committee == 'Culture & Community Facilities' or committee == 'Community Resources' or committee == '\'Bridge Builders':
        return 'Culture & Community'
    else:
        return committee

In [ ]:
pb_data['Committee'] = pb_data['Committee'].apply(rename_committees)

In [ ]:
pb_data['Committee'].value_counts(normalize=True)

In [ ]:
committee_groups = pb_data.groupby(['Committee'])
committees = committee_groups.size().to_frame(name='# of ideas submitted').reset_index()
committees.to_csv('camb-pb-data/committees.csv')

In [ ]:
committees.head()

In [ ]:
cycle_groups = pb_data.groupby('PB Cycle')
cycles = cycle_groups.size().to_frame(name="# of ideas submitted").reset_index()
cycles.to_csv('camb-pb-data/cycles.csv')

In [ ]:
cycles.head()

In [ ]:
pb_data.head()

In [ ]:
pb_by_category = committee_groups.size().to_frame(name='# of ideas').reset_index()

In [ ]:
pb_by_location = pb_data.dropna(axis=0, how='any', subset=['Latitude', 'Longitude'])

In [ ]:
# ideas by location
df_to_geojson(pb_by_location, filename='camb-pb-data/locations.geojson',
              properties=['Committee', 'PB Cycle'],
              lat='Latitude', lon='Longitude', precision=3)

In [ ]:
pb_data['Idea Submitter'].value_counts().head()

Things I am interested in finding out:
- categorize the ideas based on their content, more specific categories, rather than the few we have
- break the interest of particular types of ideas (by category and the above) by year, see if trends exist for particular types of ideas
- look at the influence of the minority, 69 (3.5% total) ideas presumably came from one person, that's a lot of influence (calculate how much based on any accepted proposals)

In [ ]:
committee_and_cycle_groups = pb_data.groupby(['Committee', 'PB Cycle'])
committees_and_cycles = committee_and_cycle_groups.size().to_frame(name='# of ideas submitted').reset_index()
committees_and_cycles.to_csv('camb-pb-data/committees_and_cycles.csv')

In [ ]:
committees_and_cycles.head()

In [ ]:
ggplot(committees_and_cycles, aes(x='PB Cycle', y='# of ideas submitted', colour='Committee')) + geom_line()

In [ ]:
# Instantiates a client
client = language.LanguageServiceClient()

In [ ]:
def entities_text(text):
    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects entities in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    entities = client.analyze_entities(document).entities

    # entity types from enums.Entity.Type
    entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
                   'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')

    entity_list = []
    for entity in entities:
        entity_list.append(entity.name)
    
    return ','.join(entity_list)

In [ ]:
pb_data['Entities'] = pb_data['Project Title'].apply(entities_text)

In [ ]:
pb_data.to_csv('camb-pb-data/pb_data_updated.csv')

In [ ]:
# method for returning frequency counts from list
def get_freq_count(list_of_entities):
    word_freq = {}
    for row in list_of_entities:
        word_array = row.split(',')
        for word in word_array:
            word = inf.singularize(word.lower().strip())
            if word in word_freq:
                word_freq[word] = word_freq[word] + 1
            else:
                word_freq[word] = 1
    return word_freq

In [ ]:
total_freq_count = get_freq_count(pb_data['Entities'])
total_words_array = list(total_freq_count.keys())
total_word_counts_array = list(total_freq_count.values())

In [ ]:
word_usage_df = pd.DataFrame.from_dict({'Word': total_words_array, 'Count': total_word_counts_array})
word_usage_df.to_csv('camb-pb-data/word_freq.csv')

In [ ]:
total_freq_count_by_cycle = {}
for name, group in cycle_groups:
    total_freq_count_by_cycle[name] = get_freq_count(group['Entities'])

In [ ]:
pb1_total_freq_count = total_freq_count_by_cycle['PB1 (October 2014-April 2015)']
pb1_total_words_array = list(pb1_total_freq_count.keys())
pb1_total_word_counts_array = list(pb1_total_freq_count.values())

In [ ]:
pb1_word_usage_df = pd.DataFrame.from_dict({'Word': pb1_total_words_array, 'Count': pb1_total_word_counts_array})
pb1_word_usage_df.to_csv('camb-pb-data/word_freq_pb1.csv')

In [ ]:
pb2_total_freq_count = total_freq_count_by_cycle['PB2 (June-December 2015)']
pb2_total_words_array = list(pb2_total_freq_count.keys())
pb2_total_word_counts_array = list(pb2_total_freq_count.values())

In [ ]:
pb2_word_usage_df = pd.DataFrame.from_dict({'Word': pb2_total_words_array, 'Count': pb2_total_word_counts_array})
pb2_word_usage_df.to_csv('camb-pb-data/word_freq_pb2.csv')

In [ ]:
pb3_total_freq_count = total_freq_count_by_cycle['PB3(June-December 2016)']
pb3_total_words_array = list(pb3_total_freq_count.keys())
pb3_total_word_counts_array = list(pb3_total_freq_count.values())

In [ ]:
pb3_word_usage_df = pd.DataFrame.from_dict({'Word': pb3_total_words_array, 'Count': pb3_total_word_counts_array})
pb3_word_usage_df.to_csv('camb-pb-data/word_freq_pb3.csv')

In [ ]:
pb4_total_freq_count = total_freq_count_by_cycle['PB4(June-December 2017)']
pb4_total_words_array = list(pb4_total_freq_count.keys())
pb4_total_word_counts_array = list(pb4_total_freq_count.values())

In [ ]:
pb4_word_usage_df = pd.DataFrame.from_dict({'Word': pb4_total_words_array, 'Count': pb4_total_word_counts_array})
pb4_word_usage_df.to_csv('camb-pb-data/word_freq_pb4.csv')

In [ ]:
pb1_trends = pb1_word_usage_df[(pb1_word_usage_df['Word'] == 'wifi') | (pb1_word_usage_df['Word'] == 'bike lane') | (pb1_word_usage_df['Word'] == 'bench') | (pb1_word_usage_df['Word'] == 'parking') | (pb1_word_usage_df['Word'] == 'housing')]
pb1_trends['PB Cycle'] = 'PB1 (Oct\'14-Apr\'15)'

In [ ]:
pb2_trends = pb2_word_usage_df[(pb2_word_usage_df['Word'] == 'wifi') | (pb2_word_usage_df['Word'] == 'bike lane') | (pb2_word_usage_df['Word'] == 'bench') | (pb2_word_usage_df['Word'] == 'parking') | (pb2_word_usage_df['Word'] == 'housing')]
pb2_trends['PB Cycle'] = 'PB2 (Jun\-Dec\'15)'

In [ ]:
pb3_trends = pb3_word_usage_df[(pb3_word_usage_df['Word'] == 'wifi') | (pb3_word_usage_df['Word'] == 'bike lane') | (pb3_word_usage_df['Word'] == 'bench') | (pb3_word_usage_df['Word'] == 'parking') | (pb3_word_usage_df['Word'] == 'housing')]
pb3_trends['PB Cycle'] = 'PB3 (Jun\-Dec\'16)'

In [ ]:
pb4_trends = pb4_word_usage_df[(pb4_word_usage_df['Word'] == 'wifi') | (pb4_word_usage_df['Word'] == 'bike lane') | (pb4_word_usage_df['Word'] == 'bench') | (pb4_word_usage_df['Word'] == 'parking') | (pb4_word_usage_df['Word'] == 'housing')]
pb4_trends['PB Cycle'] = 'PB4 (Jun\-Dec\'17)'

In [ ]:
pb_trends = pd.concat([pb1_trends, pb2_trends, pb3_trends, pb4_trends])
pb_trends = pb_trends.rename(columns={"Word": "Topic"})

In [ ]:
ggplot(pb_trends, aes(x='PB Cycle', y='Count', colour='Topic')) + geom_line()